# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [33]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
import os
import json

# Import API key
from api_keys import geoapify_key

import warnings
warnings.filterwarnings("ignore")



In [34]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data = pd.read_csv("challenge6/cities.csv")

# Display sample data
city_data.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


In [35]:
city_data.count()

City_ID       577
City          577
Lat           577
Lng           577
Max Temp      577
Humidity      577
Cloudiness    577
Wind Speed    577
Country       574
Date          577
dtype: int64

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [36]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM",alpha = 0.5, frame_width = 700, 
                                       frame_height = 500, size = "Humidity", scale = 1)

# Display the map
humidity_map 


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [37]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_narrow=city_data[(city_data['Max Temp'] < 28) &
                             (city_data['Max Temp'] > 15) &
                             (city_data['Wind Speed'] < 15) &
                             (city_data['Cloudiness'] < 90) &
                             (city_data['Humidity'] < 32)]

# Drop any rows with null values
city_data_narrow=city_data_narrow.dropna()

# Display sample data
city_data_narrow.count()

City_ID       9
City          9
Lat           9
Lng           9
Max Temp      9
Humidity      9
Cloudiness    9
Wind Speed    9
Country       9
Date          9
dtype: int64

In [38]:
city_data_narrow.head(9)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
73,73,nalut,30.3333,10.8500,25.41,27,0,5.11,LY,1666108275
102,102,rio gallegos,-51.6226,-69.2181,20.03,26,75,8.23,AR,1666108296
106,106,palencia,42.4167,-4.5000,24.40,30,40,8.66,ES,1666108219
239,239,rawson,-43.3002,-65.1023,23.74,25,2,13.52,AR,1666108102
242,242,aksu,41.1231,80.2644,15.37,20,81,3.23,CN,1666108378
256,256,prieska,-29.6641,22.7474,27.52,31,83,3.22,ZA,1666108388
258,258,comodoro rivadavia,-45.8667,-67.5000,25.92,24,0,7.20,AR,1666108389
436,436,lakki marwat,32.6079,70.9114,27.89,20,0,1.53,PK,1666108492
550,550,pali,25.7667,73.3333,27.18,26,0,1.38,IN,1666108552


### Step 3: Create a new DataFrame called `hotel_df`.

In [39]:
#Cleanup data
clean_df = city_data_narrow.dropna(axis="index", how="any")
print(len(clean_df))


9


In [40]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# Display sample data
hotel_df = clean_df[["City", "Country","Lng", "Lat", "Humidity"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(9)


,City,Country,Lng,Lat,Humidity,Hotel Name
73,nalut,LY,10.8500,30.3333,27,
102,rio gallegos,AR,-69.2181,-51.6226,26,
106,palencia,ES,-4.5000,42.4167,30,
239,rawson,AR,-65.1023,-43.3002,25,
242,aksu,CN,80.2644,41.1231,20,
256,prieska,ZA,22.7474,-29.6641,31,
258,comodoro rivadavia,AR,-67.5000,-45.8667,24,
436,lakki marwat,PK,70.9114,32.6079,20,
550,pali,IN,73.3333,25.7667,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [48]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"hotel","apikey":geoapify_key, "limit":20}


# Print a message to follow up the hotel search
print("Starting hotel search")
print('-----------------------------')

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows(): 
    city = hotel_df.loc[index,"City"] 
  
   
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"



    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} name_address: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
-----------------------------
nalut name_address: No hotel found
rio gallegos name_address: No hotel found
palencia name_address: No hotel found
rawson name_address: No hotel found
aksu name_address: No hotel found
prieska name_address: No hotel found
comodoro rivadavia name_address: No hotel found
lakki marwat name_address: No hotel found
pali name_address: No hotel found


,City,Country,Lng,Lat,Humidity,Hotel Name
73,nalut,LY,10.8500,30.3333,27,No hotel found
102,rio gallegos,AR,-69.2181,-51.6226,26,No hotel found
106,palencia,ES,-4.5000,42.4167,30,No hotel found
239,rawson,AR,-65.1023,-43.3002,25,No hotel found
242,aksu,CN,80.2644,41.1231,20,No hotel found
256,prieska,ZA,22.7474,-29.6641,31,No hotel found
258,comodoro rivadavia,AR,-67.5000,-45.8667,24,No hotel found
436,lakki marwat,PK,70.9114,32.6079,20,No hotel found
550,pali,IN,73.3333,25.7667,26,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [32]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", frame_width = 700, frame_height = 500, 
                                   size = "Hotel", scale = 1, color = "City", hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)